In [1]:
#Wordle Solver
'''
I will tackle this in the following way:
	Get wordle's word list
	Try a word, preferrably one with more vowels (I will manually input words into the browser for now)
	Input the result of the guess with alphabets (G=Green,Y=Yellow,B=Black) to denote positions and errors
	Filter words based on output:
		Keep words with green
		Keep words with same letter as yellow, then remove words with letters in same position
		Remove words with black
	Repeat process until I either get the word or I run out of tries
'''

"\nI will tackle this in the following way:\n\tGet wordle's word list\n\tTry a word, preferrably one with more vowels (I will manually input words into the browser for now)\n\tInput the result of the guess with alphabets (G=Green,Y=Yellow,B=Black) to denote positions and errors\n\tFilter words based on output:\n\t\tKeep words with green\n\t\tKeep words with same letter as yellow, then remove words with letters in same position\n\t\tRemove words with black\n\tRepeat process until I either get the word or I run out of tries\n"

In [2]:
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [ ]:
#Import word list from file
def CreateWordList(WordList):
	with open('WordList.txt','r',encoding='UTF-8') as file:
		while line := file.readline():
			WordList.append(line.rstrip())
			
#Compare guess with result
def GetStatus(status,num):
	status = list(status)
	for i in range(5):
		cell = driver.find_element(By.XPATH,f'//*[@id="wordle-app-game"]/div[1]/div/div[{num}]/div[{i+1}]/div')
		state = cell.get_attribute("data-state")
		if (state=="correct"):
			status[i] = "G"		#letter is in the correct position
		elif (state=="present"):
			status[i] = "Y"		#letter is present in the word but not placed in the correct place
		elif (state=="absent"):
			status[i] = "B"		#letter is not present in the word
	status = "".join(status)
	return status


def MakeGuess(num,status,WordList,guess):
	if num == 1:
		print(f"Guess: {(guess := 'audio')}")
		return guess
	else:
		for i in range(len(status)):
			if status[i] == 'G':	#Remove words from the list that doesn't contain this letter in ith position
				j = 0
				while (j<len(WordList)):
					if WordList[j][i] != guess[i]:
						WordList.pop(j)
					else:
						j += 1
			elif status[i] == 'Y':	#Remove words that doesn't have this letter and words which has this letter in ith position
				j = 0
				while (j<len(WordList)):
					if guess[i] not in WordList[j]:
						WordList.pop(j)
					else:
						j += 1
				j = 0
				while (j<len(WordList)):
					if WordList[j][i] == guess[i]:
						WordList.pop(j)
					else:
						j += 1
			elif status[i] == 'B':	#Remove words with this letter
				j = 0
				while (j<len(WordList)):
					if ((guess[i] in WordList[j]) and WordList[j].count(guess[i]) >= guess.count(guess[i])):
						WordList.pop(j)
					else:
						j += 1
						
		print(f"Guess: {(guess := random.choice(WordList))}")
		return guess
						

In [ ]:
WordList = []
status = "     "
guess = ""
tries = 1
chrome_options = Options()
chrome_options.page_load_strategy = "eager"
WordleURL = "https://www.nytimes.com/games/wordle/index.html"
DriverPath = "chromedriver"
service = Service(DriverPath)
driver = webdriver.Chrome(options=chrome_options, service=service)
wait = WebDriverWait(driver, 10)

driver.get(WordleURL)
print("Page Loaded")


try:
	#wait until loaded, then click
	PlayButton = wait.until(
		EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[1]/div/div/div/div[2]/div/button[3]"))
	)
	PlayButton.click()

	ClosePopupButton = wait.until(
		EC.element_to_be_clickable((By.XPATH, '//*[@id="help-dialog"]/div/div/button'))
	)
	ClosePopupButton.click()

	#wait until loaded
	Game = wait.until(
		EC.presence_of_element_located((By.XPATH, '/html/body'))
	)
except Exception as e:
	print(f"Error occured while clicking: {e}")
	exit(1)
	driver.quit()

CreateWordList(WordList)
time.sleep(2)


while (tries <= 6):
	guess = MakeGuess(tries,status,WordList,guess)
	Game.send_keys(guess)
	Game.send_keys(Keys.ENTER)
	time.sleep(2)
	status = GetStatus(status,tries)
	if status == "GGGGG":
		break
	tries += 1

time.sleep(10)
driver.quit()

Page Loaded
Guess: audio
Guess: dills
Guess: wield
